In [55]:
import pandas as pd
import numpy as np
from Files import nodes
wickelfeatures_list = nodes.nds
import coding_function as cf
import decoding_function as dec

In [2]:
def createdf(vector):
    new = np.asarray(vector)
    df = pd.DataFrame()
    df['wickelfeatures'] = wickelfeatures_list
    df['values'] = new    
    return df
    

In [3]:
df = createdf(cf.coding('#dar#'))

In [4]:
df.loc[(df['wickelfeatures'].apply(lambda x: x[2]) == '#') & (df['values'] > 0)]

,wickelfeatures,values
281,"[vowel, cont, #]",1
283,"[vowel, d1, #]",1
286,"[vowel, middle, #]",1
289,"[vowel, b2, #]",1
301,"[d2, cont, #]",1
303,"[d2, d1, #]",1
306,"[d2, middle, #]",1
309,"[d2, b2, #]",1
321,"[middle, cont, #]",1
323,"[middle, d1, #]",1


In [11]:
toma = [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [31]:
gostar = [0.019040588, 0.012052633, 0.35236615, 0.10005063, 0.32916832, 0.073083811, 0.097307645, 0.31947699, 0.27201638, 0.11171351, 0.040509794, 0.022570252, 0.83192605, 0.21355128, 0.43425792, 0.2317199, 0.1625178, 0.53424931, 0.56238937, 0.29616728, 0.035984173, 0.12696837, 0.18102938, 0.17621739, 0.1549129, 0.10456822, 0.13573548, 0.16392741, 0.19988234, 0.086747974, 0.021539299, 0.018603889, 0.10955386, 0.039493751, 0.10809998, 0.045770492, 0.068783499, 0.14489792, 0.066018559, 0.09429688, 0.094017774, 0.026666235, 0.069293737, 0.02895497, 0.14700267, 0.10910633, 0.066770062, 0.091047622, 0.11161723, 0.041579932, 0.86798382, 0.071181506, 0.11288071, 0.10100957, 0.84827143, 0.094324648, 0.71517462, 0.34046543, 0.8886947, 0.10013086, 0.031530045, 0.9209848, 0.02345353, 0.082169451, 0.87085783, 0.044378985, 0.90921474, 0.018885203, 0.86772192, 0.12760341, 0.08795578, 0.052530669, 0.033222917, 0.14583109, 0.06420096, 0.085360304, 0.10217992, 0.033955492, 0.056096494, 0.088660903, 0.28317007, 0.086705916, 0.088293783, 0.14061488, 0.21257113, 0.068725981, 0.23875168, 0.086309172, 0.26537919, 0.070963904, 0.034133635, 0.041593857, 0.12823203, 0.026042556, 0.055711742, 0.031448338, 0.069807865, 0.1591469, 0.10483999, 0.079624951, 0.29137504, 0.47079393, 0.81722987, 0.4556925, 0.95673126, 0.47869909, 0.72236592, 0.51043606, 0.83853394, 0.40154701, 0.17085943, 0.085848853, 0.19829218, 0.15600429, 0.35928556, 0.096756518, 0.13553107, 0.24392442, 0.22559015, 0.096231401, 0.72191912, 0.48811957, 0.3353925, 0.29156226, 0.7540468, 0.086364403, 0.77796918, 0.53876936, 0.88184381, 0.17101721, 0.028959228, 0.093249723, 0.068875998, 0.055027362, 0.10311768, 0.046585143, 0.065197684, 0.066155262, 0.15790512, 0.019124385, 0.0089573255, 0.32711512, 0.1561092, 0.040237959, 0.32725674, 0.038295157, 0.19204265, 0.18483019, 0.16868453, 0.037796393, 0.1062385, 0.059383228, 0.051278308, 0.074521981, 0.13107343, 0.0257475, 0.15104014, 0.076588348, 0.14730066, 0.086661614, 0.10289457, 0.073711209, 0.079122014, 0.089681275, 0.072442025, 0.050301742, 0.12921794, 0.077169701, 0.091745794, 0.054231174, 0.22307116, 0.17076525, 0.32015455, 0.14264674, 0.4834578, 0.1854659, 0.25530601, 0.19101344, 0.44735765, 0.14227714, 0.75111264, 0.03183179, 0.22328013, 0.088595293, 0.66023052, 0.1490542, 0.53371543, 0.24068177, 0.71177256, 0.052707937, 0.072808735, 0.055801883, 0.089093335, 0.067755386, 0.16402251, 0.011392612, 0.16477792, 0.070037343, 0.056234751, 0.086687587, 0.14085408, 0.064654119, 0.71046054, 0.54407954, 0.43663037, 0.25797155, 0.42749336, 0.48835298, 0.74258572, 0.39152837, 0.23941702, 0.26764831, 0.084095649, 0.13068393, 0.44747767, 0.056369737, 0.46336305, 0.3085072, 0.47259539, 0.15277115, 0.62925541, 0.58326542, 0.93903518, 0.46489292, 0.90633237, 0.1593415, 0.82809305, 0.73428524, 0.92807239, 0.1903823, 0.38796481, 0.090294234, 0.15631536, 0.19919121, 0.22906916, 0.076012395, 0.18044722, 0.32283932, 0.26177022, 0.13995609, 0.24026163, 0.31684557, 0.13479936, 0.12038258, 0.26508871, 0.078673676, 0.4033806, 0.15106326, 0.39060256, 0.24640255, 0.025798114, 0.028572569, 0.035204526, 0.036223311, 0.026215924, 0.022555616, 0.053134069, 0.062543035, 0.028772993, 0.083023563, 0.021896999, 0.020111997, 0.83566403, 0.018962929, 0.85815787, 0.019295534, 0.022905113, 0.77024984, 0.83426458, 0.026130967, 0.018189421, 0.031706825, 0.050690737, 0.019975578, 0.049492411, 0.014018185, 0.018712519, 0.060940478, 0.059689637, 0.014337421, 0.016000407, 0.024276307, 0.26239318, 0.020613946, 0.24252079, 0.010262798, 0.01176392, 0.18746342, 0.20388134, 0.015374636, 0.019140914, 0.016568998, 0.12210371, 0.014110886, 0.13342363, 0.016956069, 0.015128738, 0.14111196, 0.13193271, 0.025303366, 0.015636865, 0.017809534, 0.84676999, 0.027691685, 0.85447812, 0.021374004, 0.026303126, 0.87984318, 0.84642702, 0.021156739, 0.034806602, 0.022127889, 0.076273747, 0.017281517, 0.10344764, 0.015763745, 0.017538777, 0.082943507, 0.071262002, 0.021481544, 0.027678899, 0.016098704, 0.64426643, 0.021640593, 0.65289944, 0.016888466, 0.014880912, 0.63423616, 0.58339649, 0.022515973, 0.013102156, 0.015137771, 0.26925704, 0.015653638, 0.3118346, 0.026687432, 0.023895457, 0.27427649, 0.24145935, 0.015343634, 0.013066068, 0.02010002, 0.96004367, 0.02102061, 0.95068902, 0.017877549, 0.019886583, 0.9571144, 0.94874758, 0.022496913, 0.012369883, 0.015496778, 0.045222934, 0.014911154, 0.036228154, 0.034401651, 0.012727734, 0.046670828, 0.049734805, 0.016462129, 0.010335208, 0.030702299, 0.94730979, 0.57408935, 0.44546083, 0.21544169, 0.081743576, 0.7318269, 0.78150415, 0.14046727, 0.018978419, 0.020058788, 0.085112371, 0.085076459, 0.084303707, 0.028338231, 0.027490458, 0.11461779, 0.061599527, 0.021765664, 0.02440954, 0.070386626, 0.02438942, 0.038414437, 0.041897953, 0.014605681, 0.087780684, 0.07663177, 0.074345633, 0.026081759, 0.059302907, 0.089147337, 0.8015213, 0.31670031, 0.60394287, 0.14421599, 0.14824855, 0.53952247, 0.69250935, 0.14799194, 0.022688959, 0.024564797, 0.20747423, 0.15271096, 0.052456293, 0.022465378, 0.028444583, 0.28347477, 0.18183571, 0.020758098, 0.025389547, 0.064756535, 0.11778877, 0.047161106, 0.10360437, 0.027214842, 0.053265858, 0.22253455, 0.1877968, 0.023448452, 0.026657876, 0.036742505, 0.26604879, 0.13273728, 0.04539416, 0.095076144, 0.034436047, 0.065258175, 0.10477845, 0.052532282, 0.074429601, 0.060622193, 0.56450939, 0.36456043, 0.30774826, 0.033577565, 0.1556135, 0.62656701, 0.51188534, 0.11591686, 0.037896764, 0.08344043, 0.91412354, 0.61564654, 0.48044047, 0.18654369, 0.12070051, 0.75632364, 0.92884135, 0.061085124, 0.018709309, 0.034296136, 0.055744249, 0.045640312, 0.031230837, 0.029224774, 0.02804241, 0.075921886, 0.031921282, 0.05583306]

In [12]:
dftest = createdf(toma)

In [33]:
dfgosta = createdf(gostar)

In [39]:
dftest.loc[df['values'] > 0].head()

,wickelfeatures,values
12,"[int, vowel, cont]",0
14,"[int, d2, cont]",0
16,"[int, middle, cont]",0
19,"[int, b2, cont]",0
92,"[d1, vowel, d1]",0


In [43]:
def sixteenbest(df):
    x = df.sort_values(by = ['values'], axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')
    r = x.head(16)
    return r

In [44]:
beginning = sixteenbest(dfgosta[361:])

In [46]:
beginning

,wickelfeatures,values
362,"[#, int, vowel]",0.947310
448,"[#, b1, b1]",0.928841
442,"[#, b1, vowel]",0.914124
392,"[#, d1, vowel]",0.801521
368,"[#, int, b1]",0.781504
447,"[#, b1, back]",0.756324
367,"[#, int, back]",0.731827
398,"[#, d1, b1]",0.692509
437,"[#, back, back]",0.626567
443,"[#, b1, d1]",0.615647


In [49]:
ending = sixteenbest(dfgosta[261:360])

In [50]:
ending

,wickelfeatures,values
342,"[b1, vowel, #]",0.960044
347,"[b1, back, #]",0.957114
344,"[b1, d2, #]",0.950689
348,"[b1, b1, #]",0.948748
307,"[d2, back, #]",0.879843
264,"[int, d2, #]",0.858158
304,"[d2, d2, #]",0.854478
302,"[d2, vowel, #]",0.846770
308,"[d2, b1, #]",0.846427
262,"[int, vowel, #]",0.835664


In [52]:
y = list(beginning['wickelfeatures'])

In [62]:
dec.competion(y,2)

'o'